**Ejercicio 4 : Merton**

Utilizando información pública de la acción que ustedes decidan e información de su balance para obtener K0, calcular D, σK, para T = 1 año, y probabilidad de incumplimiento al tiempo T bajo el modelo de Merton. Calcular volatilidad anual de σA con la historia de acciones.


In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 853, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 853 (delta 53), reused 0 (delta 0), pack-reused 768
Receiving objects: 100% (853/853), 62.90 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (558/558), done.
Checking out files: 100% (413/413), done.


In [ ]:
%%R
install.packages("quantmod")
install.packages("curl")
install.packages("data.table")
install.packages("RCurl")
install.packages("bitops")
install.packages("setInternet2")
install.packages("rootSolve")
install.packages("xml2")
install.packages("mvtnorm")
library(rootSolve)
library(quantmod)
library(RCurl)
library(data.table)
library(xml2)
require(jsonlite) #para el getQuotes



R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.20.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 156640 bytes (152 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [ ]:
%%R
Symbols<-c  ("AMXL.MX")
length(Symbols)


start_date=Sys.Date()-3660 #fecha inicial

#Creación del objeto para guardar los datos
dataEnv<-new.env()

#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)


#limpiarlos, alinearnos y quedarnos con el precio de cierre

#bt.prep(dataEnv,align='remove.na',fill.gaps = T)
#stock_prices = dataEnv$prices


#muestra de datos
#Nos quedamos con los precios
stock_prices = na.omit(dataEnv$`AMXL.MX`)
stock_prices=stock_prices[,4] #Close
tail(stock_prices[,])


           AMXL.MX.Close
2022-06-13         19.63
2022-06-14         19.94
2022-06-15         19.86
2022-06-16         19.73
2022-06-17         19.79
2022-06-20         20.03


In [ ]:
%%R

getQuote <- function(ticks) {
  qRoot <- "https://query1.finance.yahoo.com/v7/finance/quote?fields=symbol,longName,regularMarketPrice,regularMarketChange,regularMarketTime,sharesOutstanding,bookValue&formatted=false&symbols="
  z <- fromJSON(paste(qRoot, paste(ticks, collapse=","), sep=""))
  z <- z$quoteResponse$result[,c("symbol", "regularMarketTime", "regularMarketPrice", "regularMarketChange", "longName", "sharesOutstanding", "bookValue" )]
  row.names(z) <- z$symbol
  z$symbol <- NULL
  names(z) <- c("Time", "Price", "Change", "Name", "Numeroacciones", "valorlibros")
  z$Time <- as.POSIXct(z$Time, origin = '1970-01-01 00:00:00')
  return(z)
}


ticks=Symbols

stqt=getQuote(Symbols)

stqt

#PARA CARGAR LOS ESTADOS FINANCIEROS Y FLUJOS DE EFECTIVO
url <- "https://finance.yahoo.com/quote/"
scrapy_stocks <- function(stock){
  if ("rvest" %in% installed.packages()) {
    library(rvest)
  }else{
    install.packages("rvest")
    library(rvest)
  }
  for (i in 1:length(stock)) {
    tryCatch(
      {


        # Balance Sheet
        bsheet <- paste0(url,stock[i],"/balance-sheet?p=",stock[i])
        wahis.session <- html_session(bsheet)
        p <-    wahis.session %>%
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>%
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric())
        Headers <- wahis.session %>%
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>%
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head(5)
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,175,4)]
        BS <- data.frame(Breakdown = Concepts[-c(1:3,11,12,22:24,32,39)], matrix(Values[!is.na(Values)], ncol = 4, byrow = T))
        names(BS) <- Headers
        temp2 <- BS



        # Shares Outstanding
        stock_url <- paste0(url,stock[i],"/key-statistics?p=",stock[i])
        SO <- html_session(stock_url) %>%
          html_nodes(xpath = '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[2]/div/div[2]/div/table/tbody/tr[3]/td[2]') %>%
          html_text() %>% gsub(pattern = "[A-z]",replacement = "") %>% as.numeric()
        temp4 <- SO

        assign(paste0(stock[i],'.f'),value = list(IS = temp1,BS = temp2,CF = temp3, SO = temp4),envir = parent.frame())

      },
      error = function(cond){
        message(stock[i], "Give error ",cond)
      }
    )
  }
}

stqt$Numeroacciones

[1] 43625500672


In [ ]:
%%R

TSE <- "Total stockholders' equity"
TL <- "Total Liabilities"
# dt <- colnames(BS)[2] # ÚLTIMA FECHA DISPONIBLE

dt="12/31/2020"
K_0=315117618*1000/1000000

D_0=1309930609*1000/1000000

#Shares Outstanding (Sexpresa en miles de millones en la página)
#En millones
#Numeroacciones=43.55*1000000000/1000000
Numeroacciones=stqt$Numeroacciones/1000000

dt=strptime(dt, "%m/%d/%Y")
dt=format(dt, "%Y-%m")
dt
#no esta sentencia entonces mi ultima fecha fue tanto
n=nrow(stock_prices[paste("/",dt,sep="")]) #NUMERO DE FILAS A ANALIZAR PARA EL ÚLTIMO DATO

A_0=stock_prices[paste("/",dt,sep="")][n] #valores actuales
print("precio actual de la acción")
print(A_0)

#A_0=A_0*stqt$Numeroacciones/1000000 #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
A_0=A_0*Numeroacciones #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
print("Ratio entre A0 y K0")
A_0/K_0

A=stock_prices[paste("/",dt,sep="")]  #VECTOR DE PRECIOS

vol_A=sqrt(var(log(as.matrix(A[2:n])/as.matrix(A[1:(n-1)]))))*sqrt(252) #VOLATILIDAD ANUAL

#Valor del CETE

r=0.055
T=1

print("A_0")
A_0
print("K_0")
K_0
print("vol_A")
vol_A
print("Deuda")
D_0
print("r")
r
print("A/K")
A_0/K_0

#[1] "Deuda"
#1309930.609
#[1] "r"
#0.055
#[1] "A/K"
 #          AMXL.MX.Close
#2020-12-31      2.002741


[1] "precio actual de la acción"
           AMXL.MX.Close
2020-12-31         14.49
[1] "Ratio entre A0 y K0"
[1] "A_0"
[1] "K_0"
[1] "vol_A"
[1] "Deuda"
[1] "r"
[1] "A/K"
           AMXL.MX.Close
2020-12-31      2.006024


In [ ]:
%%R

x=c(D_0,vol_A)

#Función para estimar vol_k y D
G = function(x)
{
  vol_k <- x[2]
  D <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}



ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

print("Deuda Final")
D=ss$root[1]
D
print("Volatilidad de valor de Empresa")
vol_k=ss$root[2]
vol_k
print("Incremento en Deuda") #D_T/D_0
ss$root[1]/D_0
print("Incremento en Volatilidad respecto a vol_A")#Vol_A/Vol_K
vol_A/ss$root[2]
#ss$f.root[2]/vol_A
print("Evaluacion en raíces")
G(c(D,vol_k))

#Probabilidad de incumplimiento
PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
print("PD MERTON")
PD_merton

# "Evaluacion en raíces"
#           AMXL.MX.Close
#2020-12-31  3.159813e+05
#2020-12-31 -4.441405e+13
#[1] "PD MERTON"
#0


[1] "Deuda Final"
[1] "Volatilidad de valor de Empresa"
[1] "Incremento en Deuda"
[1] "Incremento en Volatilidad respecto a vol_A"
[1] "Evaluacion en raíces"
[1] "PD MERTON"
[1] 0


In [ ]:
%%R

#Sólo estimar la volatilidad dada la deuda
#Probabilidad de Incumplimiento

Vk = function(x)
{
  vol_k <- x
  ((vol_k/vol_A)*(K_0/A_0))-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
}

#Definir una Deuda
D=D_0
x=vol_A #semilla de la vol_k
ss=multiroot(Vk,x,positive = TRUE) #RESOLVER EL SISTEMA.
print("vol_k")
vol_k=ss$root
vol_k
print("Ratio vol_k/vol_A")
ss$root[1]/vol_A
print("valor Vk bajo solución")
Vk(vol_k)

PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))

print("Deuda estimada")
D
print("Deuda sencilla")
D_0
print("Vol estimada")
vol_k
print("Vol semilla")
vol_A
print("PD Merton")
PD_merton

#"valor Vk bajo solución"
#           AMXL.MX.Close
#2020-12-31             0
#[1] "Deuda estimada"
#1309930.609
#[1] "Deuda sencilla"
#1309930.609
#[1] "Vol estimada"
#0
#[1] "Vol semilla"
#A matrix: 1 × 1 of type dbl
#AMXL.MX.Close
#AMXL.MX.Close	0.2501939
#[1] "PD Merton"
#1


[1] "vol_k"
[1] "Ratio vol_k/vol_A"
[1] "valor Vk bajo solución"
[1] "Deuda estimada"
[1] "Deuda sencilla"
[1] "Vol estimada"
[1] "Vol semilla"
[1] "PD Merton"
[1] 1


In [ ]:
%%R
# Eran KMV

print("A_0:")
A_0
print("Valor de K_0")
K_0
print("Valor de vol_A")
vol_A
print("Valor de Deuda Actual")
D_0
print("Tasa libre de riesgo")
r
print("Temporalidad")
T


#Función para estimar vol_k y D
Gkmv = function(x)
{
  vol_k <- x[2]
  K_0 <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
    A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

D=D_0

x=c(K_0,vol_A)

ss=multiroot(Gkmv,x,positive = TRUE) #RESOLVER EL SISTEMA.

K_e=ss$root[1]
vol_k=ss$root[2]
print("Cociente K_0/D_0")
K_e/D_0
print("Cociente vol_K/vol_A")
ss$root[2]/vol_A
print("Valor de solución en función:")
Gkmv(c(K_e,vol_k))

DI=(K_e-D)/(vol_k*K_0)
print("DI")
DI
EDF=pnorm(-DI)
print("EDF Normal")
EDF

[1] "A_0:"
[1] "Valor de K_0"
[1] "Valor de vol_A"
[1] "Valor de Deuda Actual"
[1] "Tasa libre de riesgo"
[1] "Temporalidad"
diagonal element is zero 
[1] 2
[1] "Cociente K_0/D_0"
[1] "Cociente vol_K/vol_A"
[1] "Valor de solución en función:"
[1] "DI"
[1] "EDF Normal"
[1] 0.5


**Resultados**

Acciones en circulación: 43554099200

K_0: 315117.618

Deuda final D: 1036751290994511

Deuda estimada de: 1309930.609

σK: 0

Volatilidad anual σA: 0.2501939

ED Normal: 0.500000183547989

**Ejercicio 5**

Calcular la correlación de activos, probabilidad media de incumplimiento y probabilidad conjunta de incumplmiento por el método de momento de la información del archivo “Histórico Deudores (modelo de valores de referencia).xls”.


In [ ]:
%%R

library(rootSolve) #Se carga la librería para encontrar las raices de una función multivariada
library(mvtnorm) #Se carga la librería para la distribución normal bivariada

datos = read.csv("https://drive.google.com/file/d/1Pyekz1Utkap-L_Zr4zjzaubJAqHj9AnO/view?usp=sharing") #Se lee el archivo con los datos a analizar
D_t = datos$D_t
N_t = datos$N_t

In [ ]:
%%R

#Se calcula la probabilidad media de incumplimiento.

PMI = mean(D_t/N_t)

In [ ]:
%%R
#Se calcula la probabilidad conjunta de incumplimiento.

PCI_t = (D_t*(D_t-1))/(N_t*(N_t-1))
PCI = mean(PCI_t)

In [ ]:
%%R
#Se calcula el valor crítico de referencia
VCR = qnorm(PMI)
#Se hace una estimación de la correlación de activos
correl = 0.15

#Se crea una función que reciba el valor de la correlación y calcule la funcion
#de distribución acumulada de una normal estándar bivariada



In [ ]:
%%R
G <- function(x){
  sigma = matrix(c(1,x,x,1), nrow = 2)
  pmvnorm(lower = c(-Inf, -Inf), upper=c(VCR,VCR), mean=rep(0, 2), sigma=sigma)/PCI-1
}




**Conclusión**

[1] "La probabilidad media de incumplimiento es: "

[1] 0.001004205

[1] "La probabilidad conjunta de incumplimiento es: "

[1] 1.543424e-06

[1] "El valor crítico de referencia es: "

[1] -3.088986

[1] "La correlación de activos es: "

[1] 0.03886633